#**<center>LAB-06**

###**Student id:** 202211037

###**Student Name:** Aksh Patel

In [ ]:
!pip install pyspark

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## 1. Computing full data cube lattice

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum

# Create a Spark session
spark = SparkSession.builder.appName("DataCube").getOrCreate()

# Load data from CSV files
customers = spark.read.format("csv").option("header", "true").load("/content/gdrive/MyDrive/big_data_processing/datasets/dw/widom/customers.csv")
items = spark.read.format("csv").option("header", "true").load("/content/gdrive/MyDrive/big_data_processing/datasets/dw/widom/items.csv")
sales = spark.read.format("csv").option("header", "true").load("/content/gdrive/MyDrive/big_data_processing/datasets/dw/widom/sales.csv")
stores = spark.read.format("csv").option("header", "true").load("/content/gdrive/MyDrive/big_data_processing/datasets/dw/widom/stores.csv")
# Join dataframes
data_cube = sales.join(customers, 'custid', 'inner') \
                .join(items, 'itemid', 'inner') \
                .join(stores, 'storeid', 'inner')


In [ ]:
# 1. Compute the full data cube lattice and materialize it in Parquet data format
data_cube.write.parquet("data_cube.parquet", mode='overwrite')

## 2. Performing the following operations from the materialized lattice.

###a. Show Item axis

In [ ]:
# 2. Performing the following operations from the materialized lattice

# a. Show Item axis
item_axis = data_cube.select('category').distinct()
item_axis.show()

+--------+
|category|
+--------+
|  Jacket|
|  Tshirt|
+--------+



###b. Produce roll-up of (Item, Store, Customer)

In [ ]:
# b. Produce roll-up of (Item, Store, Customer)
rollup = data_cube.groupBy('category', 'city', 'cname').agg(sum('price').alias('SalesTotal'))\
                 .withColumnRenamed('category', 'Item') \
                 .withColumnRenamed('city', 'Store') \
                 .withColumnRenamed('cname', 'Customer')
rollup.show()

+------+-------------+--------+----------+
|  Item|        Store|Customer|SalesTotal|
+------+-------------+--------+----------+
|Jacket|      Redmond|   Craig|     270.0|
|Tshirt|      Redmond|   Doris|     165.0|
|Jacket|      Seattle|     Amy|     180.0|
|Jacket|      Seattle|     Bob|     225.0|
|Tshirt|Mountain View|     Bob|     125.0|
|Tshirt|   Menlo Park|     Bob|     165.0|
|Jacket|    Palo Alto|     Bob|      80.0|
|Tshirt|      Belmont|     Amy|      65.0|
|Tshirt|    Palo Alto|   Craig|      45.0|
|Jacket|Mountain View|     Bob|      90.0|
|Tshirt|   Menlo Park|   Craig|     150.0|
|Jacket|   Menlo Park|     Bob|     165.0|
|Jacket|Mountain View|   Craig|     145.0|
|Jacket|Mountain View|   Doris|     165.0|
|Jacket|      Belmont|   Craig|      90.0|
|Jacket|    Palo Alto|   Craig|      40.0|
|Jacket|   Menlo Park|   Craig|     145.0|
|Jacket|      Redmond|   Doris|     360.0|
|Jacket|    Palo Alto|     Amy|      30.0|
|Jacket|Mountain View|     Amy|     280.0|
+------+---

### c. Show store-wise sales summary of blue Tshirt


In [ ]:
# c. Show store-wise sales summary of blue T-shirt
blue_tshirt_sales = data_cube.filter((col('category') == 'Tshirt') & (col('color') == 'blue')) \
                            .groupBy('storeid', 'city', 'category') \
                            .agg(sum('price').alias('SalesTotal'))
blue_tshirt_sales.show()

+-------+-------------+--------+----------+
|storeid|         city|category|SalesTotal|
+-------+-------------+--------+----------+
| store2|Mountain View|  Tshirt|      65.0|
| store1|    Palo Alto|  Tshirt|      70.0|
+-------+-------------+--------+----------+



### d. List all 'Tshirts' (price <= 20) sold in 'California' to young people (age < 25).

In [ ]:
# d. List all 'Tshirts' (price <= 20) sold in 'California' to young people (age < 25)
tshirts_in_ca_to_young_people = data_cube.filter((col('category') == 'Tshirt') & (col('state') == 'CA') & (col('age') < 25) & (col('price') <= 20))
tshirts_in_ca_to_young_people.show()

+-------+------+------+-----+-----+------+---+--------+-----+---------+-----------+-----+
|storeid|itemid|custid|price|cname|gender|age|category|color|     city|     county|state|
+-------+------+------+-----+-----+------+---+--------+-----+---------+-----------+-----+
| store1|item1 |cust1 |   10|  Amy|     F| 20|  Tshirt| blue|Palo Alto|Santa Clara|   CA|
| store1|item1 |cust2 |   15|  Bob|     M| 21|  Tshirt| blue|Palo Alto|Santa Clara|   CA|
| store4|item3 |cust2 |   20|  Bob|     M| 21|  Tshirt|  red|  Belmont|  San Mateo|   CA|
| store4|item3 |cust2 |   15|  Bob|     M| 21|  Tshirt|  red|  Belmont|  San Mateo|   CA|
| store4|item3 |cust2 |   10|  Bob|     M| 21|  Tshirt|  red|  Belmont|  San Mateo|   CA|
+-------+------+------+-----+-----+------+---+--------+-----+---------+-----------+-----+

